<h1>Automação Web e Busca de Informações com Python</h1>
<h2>Desafio:</h2>

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

In [11]:
# !pip install selenium

In [12]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys as k
from selenium.webdriver.common.by import By

# Passo 1: pegar a cotação do dólar

nav = wb.Chrome()

nav.get("https://www.google.com.br/")

nav.find_element(By.XPATH , 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

nav.find_element(By.XPATH , 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(k.ENTER)

cotacao_dolar = nav.find_element(By.XPATH , 
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# Passo 2: Pegar a cotação do Euro

nav.get("https://www.google.com.br/")

nav.find_element(By.XPATH , 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

nav.find_element(By.XPATH , 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').send_keys(k.ENTER)

cotacao_euro = nav.find_element(By.XPATH , 
                                 '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# Passo 3: Pegar a cotação do Ouro

nav.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = nav.find_element(By.XPATH , '//*[@id="comercial"]').get_attribute("value")

cotacao_ouro = cotacao_ouro.replace("," , ".")

print(cotacao_dolar, cotacao_euro, cotacao_ouro)

nav.quit()

5.166799999999999 5.25902738 295.60


In [13]:
# Passo 4: Importa as listas

import pandas as pd

tabela = pd.read_excel(r"C:\Users\Khawan\Downloads\Produtos.xlsx")

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [14]:
# Passo 5: Recalculando os dados das listas

# atualizando a cotação
tabela.loc[tabela["Moeda"] == "Dólar" , "Cotação"] = float(cotacao_dolar)

tabela.loc[tabela["Moeda"] == "Euro" , "Cotação"] = float(cotacao_euro)

tabela.loc[tabela["Moeda"] == "Ouro" , "Cotação"] = float(cotacao_ouro)

# atualizando o preço de compra
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizando o preço de venda
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.166800,5166.748332,1.40,7233.447665
1,Carro Renault,4500.00,Euro,5.259027,23665.623210,2.00,47331.246420
2,Notebook Dell,899.99,Dólar,5.166800,4650.068332,1.70,7905.116164
3,IPhone,799.00,Dólar,5.166800,4128.273200,1.70,7018.064440
4,Carro Fiat,3000.00,Euro,5.259027,15777.082140,1.90,29976.456066
5,Celular Xiaomi,480.48,Dólar,5.166800,2482.544064,2.00,4965.088128
6,Joia 20g,20.00,Ouro,295.600000,5912.000000,1.15,6798.800000


In [15]:
# Passo 6: Exportar as listas de dados

tabela.to_excel(r"C:\Users\Khawan\Downloads\ProdutosNovos.xlsx", index = False)